## 1. 정수 인코딩(Integer Encoding)

### 1) dictionary 사용하기

In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
raw_text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driing the barber crazy. the barber went up a huge mountain."

In [ ]:
# 문장 토큰화
sentences = sent_tokenize(raw_text)
sentences

['A barber is a person.',
 'a barber is good person.',
 'a barber is huge person.',
 'he Knew A Secret!',
 'The Secret He Kept is huge secret.',
 'Huge secret.',
 'His barber kept his word.',
 'a barber kept his word.',
 'His barber kept his secret.',
 'But keeping and keeping such a huge secret to himself was driing the barber crazy.',
 'the barber went up a huge mountain.']

In [ ]:
vocab = {}
preprocessed_sentences = []
stop_words = set(stopwords.words('english'))

for sentence in sentences:
  tokenized_sentence = word_tokenize(sentence)
  result = []

  for word in tokenized_sentence:
    word = word.lower() # 모든 단어를 소문자화하여 단어의 개수를 줄인다.
    if word not in stop_words: # 단어 토큰화 된 결과에 대해서 불용어를 제거한다.
      if len(word)>2: # 단어 길이가 2이하인 경우에 대하여 추가로 단어를 제거한다.
        result.append(word)
        if word not in vocab:
          vocab[word] = 0
        vocab[word]+=1
  preprocessed_sentences.append(result)

preprocessed_sentences

[['barber', 'person'],
 ['barber', 'good', 'person'],
 ['barber', 'huge', 'person'],
 ['knew', 'secret'],
 ['secret', 'kept', 'huge', 'secret'],
 ['huge', 'secret'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'secret'],
 ['keeping', 'keeping', 'huge', 'secret', 'driing', 'barber', 'crazy'],
 ['barber', 'went', 'huge', 'mountain']]

In [ ]:
print('단어 집합:', vocab)

단어 집합: {'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driing': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


In [ ]:
vocab_sorted = sorted(vocab.items(), key=lambda x: x[1], reverse=True)
vocab_sorted

[('barber', 8),
 ('secret', 6),
 ('huge', 5),
 ('kept', 4),
 ('person', 3),
 ('word', 2),
 ('keeping', 2),
 ('good', 1),
 ('knew', 1),
 ('driing', 1),
 ('crazy', 1),
 ('went', 1),
 ('mountain', 1)]

In [ ]:
word_to_index = {}
i=0
for (word, frequency) in vocab_sorted:
  if frequency>1: # 빈도수가 작은 단어는 제외
    i+=1
    word_to_index[word] = i

print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


In [ ]:
vocab_size = 5

# 인덱스가 5 초과인 단어 제거
word_frequency = [word for word, index in word_to_index.items() if index >= vocab_size + 1]

In [ ]:
# 해당 단어에 대한 인덱스 정보를 삭제
for w in word_frequency:
  del word_to_index[w]

word_to_index

{'barber': 1, 'huge': 3, 'kept': 4, 'person': 5, 'secret': 2}

In [ ]:
# 사전에 없는 단어들을 OOV로 정의
word_to_index['OOV'] = len(word_to_index)+1
word_to_index

{'OOV': 6, 'barber': 1, 'huge': 3, 'kept': 4, 'person': 5, 'secret': 2}

In [ ]:
encoded_sentences = []
for sentence in preprocessed_sentences:
  encoded_sentence = []
  for word in sentence:
    try:
      # 단어 집합에 있는 단어라면 해당 단어의 정수를 리턴.
      encoded_sentence.append(word_to_index[word])
    except KeyError:
      # 만약 단어 집합에 없는 단어라면 'OOV'의 정수를 리턴.
      encoded_sentence.append(word_to_index['OOV'])
  encoded_sentences.append(encoded_sentence)

encoded_sentences

[[1, 5],
 [1, 6, 5],
 [1, 3, 5],
 [6, 2],
 [2, 4, 3, 2],
 [3, 2],
 [1, 4, 6],
 [1, 4, 6],
 [1, 4, 2],
 [6, 6, 3, 2, 6, 1, 6],
 [1, 6, 3, 6]]

### 2) Counter 사용하기

In [ ]:
from collections import Counter

In [ ]:
print(preprocessed_sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driing', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [ ]:
# words = np.hstack(preprocessed_sentences)으로도 수행 가능.
all_words_list = sum(preprocessed_sentences, [])
print(all_words_list)

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driing', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


In [ ]:
vocab = Counter(all_words_list)
vocab

Counter({'barber': 8,
         'crazy': 1,
         'driing': 1,
         'good': 1,
         'huge': 5,
         'keeping': 2,
         'kept': 4,
         'knew': 1,
         'mountain': 1,
         'person': 3,
         'secret': 6,
         'went': 1,
         'word': 2})

In [ ]:
vocab_siez = 5
vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 5개의 단어만 저장
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

In [ ]:
word_to_index = {}
i = 0
for (word, frequency) in vocab:
  i+=1
  word_to_index[word] = i

word_to_index

{'barber': 1, 'huge': 3, 'kept': 4, 'person': 5, 'secret': 2}

### 3) NLTK의 FreqDist 사용하기

In [ ]:
from nltk import FreqDist
import numpy as np

In [ ]:
# np.hstack으로 문장 구분을 제거
vocab = FreqDist(np.hstack(preprocessed_sentences))
vocab

FreqDist({'barber': 8,
          'crazy': 1,
          'driing': 1,
          'good': 1,
          'huge': 5,
          'keeping': 2,
          'kept': 4,
          'knew': 1,
          'mountain': 1,
          'person': 3,
          'secret': 6,
          'went': 1,
          'word': 2})

In [ ]:
vocab_size = 5
vocab = vocab.most_common(vocab_size)
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

In [ ]:
word_to_index = {word[0] : index + 1 for index, word in enumerate(vocab)}
word_to_index

{'barber': 1, 'huge': 3, 'kept': 4, 'person': 5, 'secret': 2}

## 2. 케라스(Keras)의 텍스트 전처리

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
preprocessed_sentences = [['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]

In [ ]:
tokenizer = Tokenizer()

# fit_on_texts()안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성.
tokenizer.fit_on_texts(preprocessed_sentences)

In [ ]:
print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


In [ ]:
tokenizer.word_counts

OrderedDict([('barber', 8),
             ('person', 3),
             ('good', 1),
             ('huge', 5),
             ('knew', 1),
             ('secret', 6),
             ('kept', 4),
             ('word', 2),
             ('keeping', 2),
             ('driving', 1),
             ('crazy', 1),
             ('went', 1),
             ('mountain', 1)])

In [ ]:
tokenizer.texts_to_sequences(preprocessed_sentences)

[[1, 5],
 [1, 8, 5],
 [1, 3, 5],
 [9, 2],
 [2, 4, 3, 2],
 [3, 2],
 [1, 4, 6],
 [1, 4, 6],
 [1, 4, 2],
 [7, 7, 3, 2, 10, 1, 11],
 [1, 12, 3, 13]]

In [ ]:
vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size + 1) # 상위 5개 단어만 사용
tokenizer.fit_on_texts(preprocessed_sentences)

In [ ]:
tokenizer.word_index

{'barber': 1,
 'crazy': 11,
 'driving': 10,
 'good': 8,
 'huge': 3,
 'keeping': 7,
 'kept': 4,
 'knew': 9,
 'mountain': 13,
 'person': 5,
 'secret': 2,
 'went': 12,
 'word': 6}

In [ ]:
tokenizer.word_counts

OrderedDict([('barber', 8),
             ('person', 3),
             ('good', 1),
             ('huge', 5),
             ('knew', 1),
             ('secret', 6),
             ('kept', 4),
             ('word', 2),
             ('keeping', 2),
             ('driving', 1),
             ('crazy', 1),
             ('went', 1),
             ('mountain', 1)])

상위 5개의 단어만 사용하겠다고 선언하였는데 여전히 13개의 단어가 모두 출력  
실제 적용은 texts_to_sequences를 사용할 때 적용이 된다.

In [ ]:
tokenizer.texts_to_sequences(preprocessed_sentences)

[[1, 5],
 [1, 5],
 [1, 3, 5],
 [2],
 [2, 4, 3, 2],
 [3, 2],
 [1, 4],
 [1, 4],
 [1, 4, 2],
 [3, 2, 1],
 [1, 3]]

만약 word_index와 word_counts에서도 지정된 num_words만큼의 단어만 남기고 싶다면

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_sentences)

vocab_size = 5
words_frequency = [word for word, index in tokenizer.word_index.items() if index >= vocab_size + 1] 

# 인덱스가 5 초과인 단어 제거
for word in words_frequency:
    del tokenizer.word_index[word] # 해당 단어에 대한 인덱스 정보를 삭제
    del tokenizer.word_counts[word] # 해당 단어에 대한 카운트 정보를 삭제

print(tokenizer.word_index)
print(tokenizer.word_counts)
print(tokenizer.texts_to_sequences(preprocessed_sentences))

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}
OrderedDict([('barber', 8), ('person', 3), ('huge', 5), ('secret', 6), ('kept', 4)])
[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


In [ ]:
# 숫자 0과 OOV를 고려해서 단어 집합의 크기는 +2
vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size + 2, oov_token = 'OOV')
tokenizer.fit_on_texts(preprocessed_sentences)

In [ ]:
print('단어 OOV의 인덱스 : {}'.format(tokenizer.word_index['OOV']))

단어 OOV의 인덱스 : 1


In [ ]:
tokenizer.texts_to_sequences(preprocessed_sentences)

[[2, 6],
 [2, 1, 6],
 [2, 4, 6],
 [1, 3],
 [3, 5, 4, 3],
 [4, 3],
 [2, 5, 1],
 [2, 5, 1],
 [2, 5, 3],
 [1, 1, 4, 3, 1, 2, 1],
 [2, 1, 4, 1]]